In [1]:
from agents import State,CustomAgent
import asyncio
import nest_asyncio
import json
from tools import *

In [2]:
questions=get_question()

In [3]:
async def process_question(index, row, semaphore):
    async with semaphore:
        try:
            if row['answer'] == '' and int(row['ID'])<=256 and int(row['ID'])>0:
                question = row['question']
                qes_type = row['problem_type']
                print(f"处理 id 为 {row['ID']} 问题类型为 {row['problem_type']} 的问题：{question}")
                state=State(
                    message=[],
                    question=question,
                    question_type=qes_type,
                    o1_count=0,
                    doc='',
                    code_ans='',
                    code='',
                    doc_keywords='',
                    point=0,
                    reflect_reason='',
                    is_verify=False,
                    verbose=False
                    )
                agent=CustomAgent(state)
                state=await agent.arun()
            if questions.iloc[index]['answer'] == '' and int(row['ID'])<=256 and int(row['ID'])>0:
                answer=state['answer']
                code=state['code']
                is_verify=state['is_verify']
                questions.loc[index, 'answer'] = str(answer)
                questions.loc[index, 'code'] = str(code)
                questions.loc[index, 'is_verify'] = str(is_verify)
                print(f"处理 id 为 {row['ID']} 的代码：{code}")
                print(f"处理 id 为 {row['ID']} 的答案：{answer}")
                data_list = questions.to_dict(orient='records')
                json_str = json.dumps(data_list, separators=(',', ':'),ensure_ascii=False)
                formatted_json_str = json_str.replace('},{', '},\n{')

                with open('answer/Final_Example.json', 'w', encoding='utf-8') as f:
                    f.write(formatted_json_str)
                    f.close()
        except Exception as e:
            print(f"处理 id 为 {row['ID']} 的问题时出现错误：{e}")

In [4]:
async def main():
    semaphore = asyncio.Semaphore(20)  # 限制最多5个任务同时运行
    tasks = [process_question(index, row, semaphore) for index, row in questions.iterrows()]
    await asyncio.gather(*tasks)

In [ ]:

# 允许在已有的事件循环中再次运行事件循环
nest_asyncio.apply()
# 替换asyncio.run(main())为以下代码
async def main_wrapper():
    await main()

# 获取当前的事件循环并运行
loop = asyncio.get_event_loop()
loop.run_until_complete(main_wrapper())